In [77]:
import cnvpytor
import subprocess

print(cnvpytor.__version__)

1.3.1


In [78]:
filename = "test_with_del__3_formated.fa"
command = f"cat {filename} | wc -l"
result = subprocess.check_output(command, shell=True)
line_count = int(result.strip())
line_count

292422

In [79]:
read_count = line_count * 50 * 30 / 202
read_count = round(read_count)
read_count

2171450

In [80]:
command = f"wgsim -d 300 -N {read_count} -1 101 -2 101 {filename} frag1Copy_test.fastq frag2Copy_test.fastq"
result = subprocess.check_output(command, shell=True)
print(result)

[wgsim] seed = 1706544310
[wgsim_core] calculating the total length of the reference sequence...
[wgsim_core] 1 sequences, total length: 14621005


b'chr22\t525\tC\tM\t+\nchr22\t649\tC\tS\t+\nchr22\t1997\tG\tC\t-\nchr22\t2899\tG\tS\t+\nchr22\t3489\tC\tY\t+\nchr22\t3803\tC\tM\t+\nchr22\t3829\tT\tK\t+\nchr22\t3868\tC\tM\t+\nchr22\t4202\tA\tR\t+\nchr22\t6194\tG\tR\t+\nchr22\t6471\t-\tCA\t+\nchr22\t7094\tC\tY\t+\nchr22\t7139\tG\tR\t+\nchr22\t7781\tA\tT\t-\nchr22\t9845\tA\tC\t-\nchr22\t9997\tA\tW\t+\nchr22\t10381\tC\tM\t+\nchr22\t12717\t-\tC\t+\nchr22\t13321\tG\tR\t+\nchr22\t14817\tA\tR\t+\nchr22\t20195\tC\tM\t+\nchr22\t22042\tT\tK\t+\nchr22\t22302\tCC\t-\t-\nchr22\t22761\tC\tY\t+\nchr22\t23694\tA\tT\t-\nchr22\t24746\tG\tC\t-\nchr22\t26155\tC\tS\t+\nchr22\t26552\tG\tR\t+\nchr22\t28377\tC\tG\t-\nchr22\t28919\tT\tY\t+\nchr22\t31588\tT\tY\t+\nchr22\t32592\t-\tG\t+\nchr22\t34256\tA\tW\t+\nchr22\t36383\tA\tC\t-\nchr22\t36570\tT\t-\t-\nchr22\t37174\tG\tC\t-\nchr22\t37726\tT\tY\t+\nchr22\t37907\tA\tW\t+\nchr22\t38460\tA\tW\t+\nchr22\t39391\tA\tG\t-\nchr22\t40422\tG\tK\t+\nchr22\t44974\tA\tG\t-\nchr22\t45907\tC\tS\t+\nchr22\t46584\tC\tS\t+\nch

In [81]:
# bowtie related stuff
!bowtie2-build reference.fa bias
!bowtie2 -p 4 -k 15 --no-mixed -x bias -1 frag1Copy_test.fastq -2 frag2Copy_test.fastq -S biasOut.sam

Settings:
  Output files: "bias.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  reference.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:01
bmax according to bmaxDivN setting: 12826141
Using parameters --bmax 9619606 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 9619606 --dcv 1024
Constructing suffix-array element

In [82]:
# sam to bam file conversion
!samtools sort biasOut.sam -o biasOut_sorted.bam
!samtools index biasOut_sorted.bam

[bam_sort_core] merging from 2 files and 1 in-memory blocks...


In [83]:
app = cnvpytor.Root("test.pytor", create=True, max_cores=8)

In [84]:
app.rd(["biasOut_sorted.bam"])

In [85]:
app.calculate_histograms([100, 1000, 10000, 100000, 1000000, 10000000])

In [86]:
app.partition([100, 1000, 10000, 100000, 1000000, 10000000])

In [87]:
calls = app.call([100, 1000, 10000, 100000, 1000000, 10000000], print_calls=True)

duplication	chr22:643201-644800	1600	1.4471	1.638788e-05	9.881072e-64	1.000000e+00	1.000000e+00	0.0000	1.0000	0
duplication	chr22:1306101-1325300	19200	1.3590	0.000000e+00	1.218033e-173	0.000000e+00	5.900472e-229	0.0000	1.0000	0
duplication	chr22:2926201-2963000	36800	3.0514	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:4546401-4713300	166900	1.6227	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:6655001-6661800	6800	2.3658	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:6848301-6854500	6200	1.6330	0.000000e+00	2.361065e-260	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:8039401-8047700	8300	1.9052	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:9424001-9461600	37600	1.4479	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:13230601-13395900	165300	1.3709	0.000000e+00	0.000000e+00	0.0000

In [88]:
for bin_size in calls:
    print("Bin size: ", bin_size)

Bin size:  100
Bin size:  1000
Bin size:  10000
Bin size:  100000
Bin size:  1000000
Bin size:  10000000


In [89]:
print(f"{filename=}, Histogram Size: 100")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__3_formated.fa', Histogram Size: 100
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,643201,644800,1600,1.4471396427446177,0.0
duplication,chr22,1306101,1325300,19200,1.3589700594107716,0.0
duplication,chr22,2926201,2963000,36800,3.051374560118751,0.0
duplication,chr22,4546401,4713300,166900,1.6227065898287303,0.0
duplication,chr22,6655001,6661800,6800,2.365778146863425,0.0
duplication,chr22,6848301,6854500,6200,1.633023619729193,0.0
duplication,chr22,8039401,8047700,8300,1.905213253920726,0.0
duplication,chr22,9424001,9461600,37600,1.4478790955549627,0.0
duplication,chr22,13230601,13395900,165300,1.370883795924192,0.0
duplication,chr22,13480901,13512800,31900,4.12777045251534,0.0
duplication,chr22,15242601,15246400,3800,2.9395748826520633,0.0
duplication,chr22,18662501,18664600,2100,1.480431306572258,0.0
duplication,chr22,18664801,18665900,1100,1.5183208624572664,0.0
duplication,chr22,18680801,18682700,1900,1.4547411305244078,0.0
duplication,

In [90]:
print(f"{filename=}, Histogram Size: 1000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__3_formated.fa', Histogram Size: 1000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,1306001,1325000,19000,1.3547585568203033,0.0
duplication,chr22,2926001,2963000,37000,3.028287009385069,0.0
duplication,chr22,4546001,4713000,167000,1.615119886161519,0.0
duplication,chr22,6655001,6662000,7000,2.3214474524741147,0.0
duplication,chr22,8040001,8048000,8000,1.889407536737982,0.0
duplication,chr22,9424001,9462000,38000,1.4377362227210686,0.0
duplication,chr22,13231001,13411000,180000,1.3354645364977693,0.0
duplication,chr22,13481001,13513000,32000,4.09863108691042,0.0
duplication,chr22,18705001,18881000,176000,2.799287688784072,0.00014095887552554353
duplication,chr22,19259001,19268000,9000,3.2835287079766604,0.0
duplication,chr22,20634001,20655000,21000,1.3836333807070411,0.00028363430084144843
duplication,chr22,21248001,21382000,134000,1.5211669574467168,0.00014259364639680378
duplication,chr22,21485001,21505000,20000,1.414738787720055,9.6979101

In [91]:
print(f"{filename=}, Histogram Size: 10,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__3_formated.fa', Histogram Size: 10,000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,4550001,4710000,160000,1.60252963085515,0.0
duplication,chr22,13230001,13390000,160000,1.3533182064157503,0.0
duplication,chr22,18710001,18880000,170000,2.7758157290938983,0.00014547970500839659
duplication,chr22,21250001,21380000,130000,1.5080096544422006,0.00014107471805403703
duplication,chr22,21530001,21670000,140000,1.355085302619647,0.0004606205013086456
duplication,chr22,24200001,24240000,40000,2.547277680458833,4.703619696668792e-05
duplication,chr22,25150001,25310000,160000,1.2974143627874626,0.00013440807596796651
duplication,chr22,28470001,28600000,130000,3.0793230797509836,4.2746710571675204e-05
duplication,chr22,30800001,30980000,180000,2.968396460455788,2.5984630091300994e-05
duplication,chr22,31150001,31360000,210000,1.4478236486798028,5.143403407770796e-05
duplication,chr22,31870001,32250000,380000,1.2944469177234916,9.51253248502191e-05
dup

In [92]:
print(f"{filename=}, Histogram Size: 100,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__3_formated.fa', Histogram Size: 100,000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,45700001,51300000,5600000,1.9046218099668737,7.328705573148778e-05


In [93]:
print(f"{filename=}, Histogram Size: 1,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__3_formated.fa', Histogram Size: 1,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [94]:
print(f"{filename=}, Histogram Size: 10,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__3_formated.fa', Histogram Size: 10,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [95]:
del app